In [ ]:
import sqlite3
import pandas as pd

# Подключение к базе данных Sqlite
conn = sqlite3.connect('./data/finance.lite')

# Чтение таблицы 'transactions' в DataFrame
v_df = pd.read_sql_query("SELECT * FROM trans_all", conn)

# Закрытие соединения с базой данных
conn.close()

v_df.date = pd.to_datetime(v_df.date, format='%Y-%m-%d %H:%M:%S')

# Вывод первых нескольких строк DataFrame для проверки
# v_df.iloc[9513:9518]
# v_df.info()
# v_df.loc[v_df.payment == 0, 'category'].unique()
v_df.date.map(type).value_counts()




In [ ]:
v_df_month_in = (v_df
                # .loc[(v_df['payment'] == 1)]
                .loc[(v_df['payment'] == 0) & ((v_df['category'] == 'Зарплата Д') | (v_df['category'] == 'Зарплата')) & (v_df['date'] >= '2022-01-01')]
                # .groupby(['category', pd.Grouper(key='date', freq='M')])
              .groupby(['category', v_df['date'].dt.to_period('M')])
              ['amount']
              .sum()
              .reset_index()
              )
# print(v_df_month_in.values)
# v_df_month_in.info()
median_per_year = v_df_month_in.groupby(v_df_month_in['date'].dt.year)['amount'].median()

In [ ]:
# Первичная настройка
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Set the width and height of the figure
plt.figure(figsize=(20,6))

# Line chart showing how FIFA rankings evolved over time 
# sns.lineplot(data=v_df_month_in)

ax = sns.barplot(
    data=v_df_month_in,
    x='date',       # ось X — месяцы
    y='amount',     # ось Y — значения
    # hue='category', # если хочешь разбить по категориям
)

# Подписи осей
ax.set_xlabel("Год и Месяц")    # название оси X
ax.set_ylabel("Сумма, тыс.руб") # название оси Y

# Форматируем ось Y в тысячах
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x/1000)}'))

# Добавляем горизонтальные линии на каждом тике Y
yticks = ax.get_yticks()  # получаем значения делений оси Y
for y in yticks:
    ax.axhline(y=y, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

# Подписываем значения над каждым баром
for p in ax.patches:
    height = p.get_height()
    ax.text(
        x=p.get_x() + p.get_width()/2,  # по центру бара
        y=height + 10000,                # чуть выше столбца
        s=f'{height:.0f}',              # текст — значение с без знаками после запятой
        ha='center'
    )

plt.xticks(rotation=45)  # наклон меток X, чтобы было читаемо
plt.show()
